# prj1 주제 : 쇼핑몰의 정보를 분석(평점,가격등)
## 타깃 : 네이버쇼핑
### 순서
- 1.크롤러제작
 

### 1. 크롤러 제작

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')

In [41]:
query = "삼성"

titles = [] #제품명
prices = []
review_counts = [] #리뷰수
buy_counts = [] # 구매건수
published_dates = []
favorites = [] #찜하기

driver = Chrome('./chromedriver')

def navshop(query):
    for page_no in tqdm(range(1, 6)):

        page_url = f"https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery={query}&pagingIndex={page_no}&pagingSize=40&productSet=total&query={query}&sort=rel&timestamp=&viewType=list"
        driver.get(page_url)
        time.sleep(1)
        
        # scrolling하는 코드
        for scroll_down in range(7):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(0.5)

        list_basis = driver.find_element(By.CLASS_NAME,"list_basis")
        item_list = list_basis.find_elements(By.CLASS_NAME,'basicList_inner__xCM3J')

        items = []
        for i in tqdm(range(len(item_list))):
            item = item_list[i]
            title = item.find_element(By.CLASS_NAME,'basicList_title__VfX3c')
            titles.append(title.text)


            price = item.find_element(By.CLASS_NAME,"price_num__S2p_v").text[:-1].replace(',', '')
            prices.append(price)

            footer = item.find_element(By.CLASS_NAME,'basicList_etc_box__5lkgg')#.text
            reviews = footer.find_elements(By.CLASS_NAME,'basicList_num__sfz3h')
            footer_text = footer.text
            try:
                if "구매건수" in footer_text:
                    review_counts.append(int(reviews[0].text.replace(',', '')))
                    buy_counts.append(int(reviews[1].text.replace(',', '')))
                    favorites.append(int(reviews[2].text.replace(',', '')))
                    date = footer.find_elements(By.TAG_NAME,'span')[0].text[4:]
                    published_dates.append(date)
                else:
                    #일부코드 수정(인덱싱을 0부터)
                    favorites.append(int(reviews[0].text.replace(',', '')))
                    review_counts.append(int(reviews[0].text.replace(',', '')))
                    date = footer.find_elements(By.CLASS_NAME,'basicList_etc__LSkN_')[0].text[4:]
                    published_dates.append(date)

            except IndexError: ## 리뷰가 아직 충분하지 않아서, 정보가 안뜨는 케이스
                review_counts.append(0)
                favorites.append(int(reviews[0].text.replace(',', '')))
                date = footer.find_elements(By.CLASS_NAME,'span')[0].text[4:]
                published_dates.append(date)
        return print("완료")
result = pd.DataFrame({"제품명" : titles,
                       "가격" : prices,
                       "리뷰수" : review_counts,
                       "등록일" : published_dates,
                       "찜하기" : favorites})
result

,제품명,가격,리뷰수,등록일,찜하기


In [10]:
items

[]

In [30]:
print(len(titles), len(prices), len(review_counts), len(published_dates), len(favorites))

206 206 206 206 206


In [22]:
result = pd.DataFrame({"제품명" : titles,
                       "가격" : prices,
                       "리뷰수" : review_counts,
                       "등록일" : published_dates,
                       "찜하기" : favorites})
result

,제품명,가격,리뷰수,등록일,찜하기
0,삼성전자 삼성 SPA-NMA1PMB,24500,144,2020.12.,144
1,거보아이엔씨 홈마블 SH20,44820,104,4.9104,104
2,삼성전자 삼성 SPA-KMA3PBW,15200,19,2019.01.,19
3,삼성전자 삼성 SKP-1000B,19900,6,2017.11.,6
4,삼성전자 삼성 갤럭시Z 플립4 5G 256GB [자급제],1035920,1281,"4.91,281",1281
...,...,...,...,...,...
201,삼성전자 삼성 갤럭시Z 폴드4 5G 512GB [KT],1097800,16,2022.08.,16
202,삼성전자 삼성 갤럭시Z 플립4 5G 256GB [KT],305500,57,2022.08.,57
203,삼성전자 삼성 갤럭시Z 플립4 5G 512GB [KT],424100,37,2022.08.,37
204,삼성전자 삼성 CLT-Y510S (정품),59570,513,4.8513,513


In [18]:
result.to_excel("g:/dataAnal/dataset/navershop.xlsx", index=False)

### 날짜 인덱싱 추후에 수정예정, 나머지는 제대로 동작함!
- 결점 : 중간에 리뷰수가 안맞는다, 네이버자체의 문제라 pass
- 결점수정완료 인덱싱과 슬라이싱 시작을 어디서부터 하느냐의 문제